In [15]:
import json
import pandas as pd
import numpy as np

In [24]:
enriched_log = '../../data/MovieSummaries/v2_movie_processed_enriched.json'
movies_freebase = '../../data/MovieSummaries/movies_freebase.json'

In [25]:
with open(enriched_log) as file:
    data = json.load(file)

with open(movies_freebase) as file:
    freebase = json.load(file)

In [26]:
data = pd.DataFrame(data)
data.drop_duplicates(inplace=True)
freebase = pd.DataFrame(freebase)

In [43]:
merged = pd.merge(data, freebase[['fb_movie_id', 'discription']], left_on='Freebase movie ID', right_on='fb_movie_id', suffixes=('', 'freebase'), how='left')
merged.head(2)

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie box office revenue,Movie runtime,languages,movie_release_date,countries_old,countries,genres,wiki_api_title,wikipedia_movie_id,release_date,plot_summary,keywords,cast,page_summary,page,fb_movie_id,discription
0,975900,/m/03vyhn,Ghosts of Mars,14010832.0,98.0,['english'],2001-08-24,['United States of America'],['United States of America'],None,Ghosts of Mars,Ghosts of Mars,NaN,"Set in the year 2176, Mars has been 85% terraf...",NaN,None,Ghosts of Mars (titled onscreen as John Carpen...,Ghosts of Mars (titled onscreen as John Carpen...,/m/03vyhn,John Carpenter's Ghosts of Mars is a 2001 Amer...
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,NaN,95.0,['english'],2000-02-16,['United States of America'],['United States of America'],None,Getting Away with Murder: The JonBenet Ramsey ...,Getting Away with Murder: The JonBenet Ramsey ...,NaN,"On December 25, 1996, in Boulder, Colorado, an...",NaN,Alice Barrett-Mitchell as Reporter\nCliff DeYo...,Getting Away with Murder: The JonBenet Ramsey ...,Getting Away with Murder: The JonBenet Ramsey ...,/m/08yl5d,Getting Away with Murder: The JonBenet Ramsey ...


In [49]:
def make_summary(row):
    result = ""
    if isinstance(row['cast'], str):
        result += f"Cast: {row['cast']}" + '\n'

    plots_count = 0
    if isinstance(row['plot_summary'], str) and row['plot_summary'] != row['page']:
        result += row['plot_summary'] + '\n'
        plots_count += 1

    if isinstance(row['discription'], str) and plots_count < 2:
        result += row['discription'] + '\n'
        plots_count += 1

    if isinstance(row['page_summary'], str)  and plots_count < 2:
        result += row['page_summary'] + '\n'
        plots_count += 1

    if isinstance(row['page'], str)  and plots_count < 2:
        result += row['page'] + '\n'
        plots_count += 1
    return None if len(result) == 0 else result


merged['summary'] = merged.apply(make_summary, axis=1)

In [50]:
merged['summary'].isna().sum()

np.int64(9034)

In [51]:
movie_to_summary = dict()
for i, row in merged.iterrows():
    if row['summary'] is None:
        continue
    if row['Freebase movie ID'] in movie_to_summary and len(movie_to_summary[row['Freebase movie ID']]) < len(row['summary']):
        continue
    movie_to_summary[row['Freebase movie ID']] = dict(
        summary=row['summary'],
        name=row['Movie name']
    )

In [52]:
with open('../../data/MovieSummaries/summaries.json', 'w') as file:
    json.dump(movie_to_summary, file, indent=2)

In [53]:
len(movie_to_summary)

79589